In [1]:
from __future__ import print_function
from openpathsampling.experimental.storage import Storage, ops_class_info, ops_schema
from openpathsampling.experimental.simstore import SQLStorageBackend
from openpathsampling.experimental.simstore import StorageTable
from openpathsampling.experimental.simstore.serialization_helpers import get_uuid

import collections
import numpy as np

In [2]:
import logging
import sys

root = logging.getLogger()
root.setLevel(logging.DEBUG)

ch = logging.StreamHandler(sys.stdout)
ch.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
root.addHandler(ch)

In [3]:
backend = SQLStorageBackend("test.sql", mode='w')
storage = Storage.from_backend(
    backend=backend,
    schema=ops_schema,
    class_info=ops_class_info
)

2020-05-19 10:06:02,437 - openpathsampling.experimental.storage.sql_backend - INFO - Add schema table uuid
2020-05-19 10:06:02,438 - openpathsampling.experimental.storage.sql_backend - INFO - Add schema table tables
2020-05-19 10:06:02,444 - openpathsampling.experimental.storage.sql_backend - INFO - Add schema table samples
2020-05-19 10:06:02,451 - openpathsampling.experimental.storage.sql_backend - INFO - Add schema table sample_sets
2020-05-19 10:06:02,456 - openpathsampling.experimental.storage.sql_backend - INFO - Add schema table trajectories
2020-05-19 10:06:02,463 - openpathsampling.experimental.storage.sql_backend - INFO - Add schema table move_changes
2020-05-19 10:06:02,483 - openpathsampling.experimental.storage.sql_backend - INFO - Add schema table steps
2020-05-19 10:06:02,489 - openpathsampling.experimental.storage.sql_backend - INFO - Add schema table details
2020-05-19 10:06:02,504 - openpathsampling.experimental.storage.sql_backend - INFO - Add schema table simulation

In [4]:
import openpathsampling as paths
logging.getLogger('openpathsampling').setLevel(logging.CRITICAL)
old_storage = paths.Storage("../toy_mstis_1k_OPS1_py36.nc", 'r')

In [5]:
sample_set = old_storage.samplesets[-1]

In [6]:
storage.save(sample_set)

In [7]:
storage.sync_all()

In [8]:
samples = StorageTable(storage, 'samples')

In [9]:
logging.getLogger('openpathsampling.experimental.storage').setLevel(logging.INFO)
#logging.getLogger('storage').setLevel(logging.INFO)
#logging.getLogger('serialization').setLevel(logging.INFO)
for s in samples:
    print(s)

Sample(RepID: 8, Ens: <openpathsampling.ensemble.MinusInterfaceEnsemble object at 0x117818b70>, Trajectory[150])
Sample(RepID: 3, Ens: <openpathsampling.ensemble.MinusInterfaceEnsemble object at 0x11ac9fda0>, Trajectory[102])
Sample(RepID: -3, Ens: <openpathsampling.ensemble.TISEnsemble object at 0x11b14a908>, Trajectory[6])
Sample(RepID: 2, Ens: <openpathsampling.ensemble.TISEnsemble object at 0x11ac9fd30>, Trajectory[62])
Sample(RepID: 5, Ens: <openpathsampling.ensemble.TISEnsemble object at 0x117573390>, Trajectory[115])
Sample(RepID: 1, Ens: <openpathsampling.ensemble.MinusInterfaceEnsemble object at 0x11b772940>, Trajectory[554])
Sample(RepID: 7, Ens: <openpathsampling.ensemble.TISEnsemble object at 0x11b7921d0>, Trajectory[164])
Sample(RepID: 0, Ens: <openpathsampling.ensemble.TISEnsemble object at 0x11b7d0a90>, Trajectory[4])
Sample(RepID: 4, Ens: <openpathsampling.ensemble.TISEnsemble object at 0x11b7d09e8>, Trajectory[28])
Sample(RepID: -2, Ens: <openpathsampling.ensemble.TISE

In [10]:
s5 = samples[5]
print(s5)

Sample(RepID: 1, Ens: <openpathsampling.ensemble.MinusInterfaceEnsemble object at 0x11b772940>, Trajectory[554])


In [11]:
s5_direct = storage.samples[5]

In [12]:
# importantly, is: same object in memory, we don't rebuild it
assert s5 is s5_direct

In [13]:
len(s5.trajectory)

554

In [14]:
print(s5)

Sample(RepID: 1, Ens: <openpathsampling.ensemble.MinusInterfaceEnsemble object at 0x11b772940>, Trajectory[554])


In [15]:
s5.trajectory

Trajectory[554]

In [16]:
len(s5.trajectory)

554

In [17]:
s5.trajectory.xyz[:10]

array([[[-0.39249977, -0.3338929 ]],

       [[-0.3855248 , -0.33292955]],

       [[-0.3801172 , -0.332006  ]],

       [[-0.37458295, -0.3321976 ]],

       [[-0.3682818 , -0.33326316]],

       [[-0.3603947 , -0.3338907 ]],

       [[-0.35361275, -0.33607066]],

       [[-0.34789062, -0.3403274 ]],

       [[-0.34188282, -0.3455238 ]],

       [[-0.33531275, -0.3517007 ]]], dtype=float32)

In [18]:
# TODO: this appears to be a problem with lazy-loading
s5.trajectory.to_dict()

{'snapshots': [<openpathsampling.engines.toy.snapshot.ToySnapshot at 0x11bd08f60>,
  <openpathsampling.engines.toy.snapshot.ToySnapshot at 0x11bcfd0f0>]}

In [19]:
samples[0]

<Sample @ 0x11bc29f60>

In [20]:
len(samples)

13

In [21]:
logging.getLogger('sql_backend').setLevel(logging.DEBUG)
logging.getLogger('storage').setLevel(logging.DEBUG)
logging.getLogger('serialization').setLevel(logging.DEBUG)

samples.save(samples[0])

In [22]:
len(storage.trajectories)

13

In [23]:
len(storage.steps)

0

In [24]:
print(storage.summary())

File: test.sql
Includes tables:
* samples: 13 items
* sample_sets: 1 items
* trajectories: 13 items
* move_changes: 0 items
* steps: 0 items
* details: 0 items
* simulation_objects: 411 items
* snapshot0: 1374 items



### Make simulation objects into sub-tables (e.g., for `volumes`, etc)

In [25]:
print(storage.summary(detailed=True))

File: test.sql
Includes tables:
* samples: 13 items
* sample_sets: 1 items
* trajectories: 13 items
* move_changes: 0 items
* steps: 0 items
* details: 0 items
* snapshot0: 1374 items
* volumes: 72 items
* ensembles: 326 items
* pathsimulators: 0 items
* pathmovers: 0 items
* networks: 0 items
* cvs: 3 items
* engines: 1 items
* misc_simulation: 9 items



In [26]:
for (obj) in storage.misc_simulation:
    print(obj)

Gaussian(-0.7000000000000001, [12. 12.], [ 0.5 -0.5])
Gaussian(-0.7000000000000001, [12. 12.], [-0.5 -0.5])
OuterWalls([1. 1.], [0. 0.])
Gaussian(-0.7000000000000001, [12. 12.], [0.  0.4])


In [27]:
storage.cvs

In [28]:
storage.volumes

In [29]:
# TODO: pseudo-table should be dict-like, but not dict. Should:
# 1. give len as number of unique items
# 2. iter over unique items

In [30]:
storage._storage_tables

{'samples': <openpathsampling.experimental.storage.storage.StorageTable at 0x11b7d6d30>,
 'sample_sets': <openpathsampling.experimental.storage.storage.StorageTable at 0x117522f98>,
 'trajectories': <openpathsampling.experimental.storage.storage.StorageTable at 0x11b94fac8>,
 'move_changes': <openpathsampling.experimental.storage.storage.StorageTable at 0x11b94fb38>,
 'steps': <openpathsampling.experimental.storage.storage.StorageTable at 0x11b94fb00>,
 'details': <openpathsampling.experimental.storage.storage.StorageTable at 0x11b94fcf8>,
 'simulation_objects': <openpathsampling.experimental.storage.storage.StorageTable at 0x11b94fa58>,
 'snapshot0': <openpathsampling.experimental.storage.storage.StorageTable at 0x11b94fb70>}

In [31]:
len(storage.volumes)

72

In [32]:
len(storage.volumes._uuid_to_obj)

72

In [33]:
for k, v in list(storage.volumes._uuid_to_obj.items())[:10]:
    print(k, repr(v))

174395458874316017953765140704788480036 <openpathsampling.volume.CVDefinedVolume object at 0x11ac9f7f0>
205825572803023860022845415596767576760 <openpathsampling.volume.CVDefinedVolume object at 0x11ac9f780>
174395458874316017953765140704788480034 <openpathsampling.volume.CVDefinedVolume object at 0x11b14af28>
205825572803023860022845415596767576750 <openpathsampling.volume.CVDefinedVolume object at 0x11ac9fd68>
205825572803023860022845415596767576744 <openpathsampling.volume.CVDefinedVolume object at 0x11b772b38>
205825572803023860022845415596767576754 <openpathsampling.volume.CVDefinedVolume object at 0x11b77dd30>
174395458874316017953765140704788480032 <openpathsampling.volume.CVDefinedVolume object at 0x11b7698d0>
205825572803023860022845415596767576740 <openpathsampling.volume.CVDefinedVolume object at 0x11b78ad30>
205825572803023860022845415596767576764 <openpathsampling.volume.CVDefinedVolume object at 0x11b792208>
205825572803023860022845415596767576752 <openpathsampling.volume

In [34]:
for k, v in list(storage.volumes._name_to_uuid.items()):
    print(k, v)

C 174395458874316017953765140704788480036
B 174395458874316017953765140704788480034
A 174395458874316017953765140704788480032
all states except C 205825572803023860022845415596767577134
all states except A 205825572803023860022845415596767576786
all states except B 205825572803023860022845415596767576960


In [35]:
storage.volumes['A']

In [36]:
storage.close()

# Opening a closed storage

In [37]:
backend = SQLStorageBackend("test.sql", mode='r')
storage = Storage.from_backend(
    backend=backend,
    schema=ops_schema,
    class_info=ops_class_info
)

2020-05-19 10:06:23,111 - openpathsampling.experimental.storage.storage - INFO - Missing info from 0 dynamically-registered tables
2020-05-19 10:06:23,112 - openpathsampling.experimental.storage.ops_storage - INFO - Found 0 possible lookups
2020-05-19 10:06:23,113 - openpathsampling.experimental.storage.ops_storage - INFO - Lookups for tables: dict_keys([])
2020-05-19 10:06:23,113 - openpathsampling.experimental.storage.storage - INFO - Successfully registered 0 missing tables


In [38]:
print(storage.summary())

File: test.sql
Includes tables:
* samples: 13 items
* sample_sets: 1 items
* trajectories: 13 items
* move_changes: 0 items
* steps: 0 items
* details: 0 items
* simulation_objects: 411 items
* snapshot0: 1374 items



In [39]:
storage._storage_tables

{'samples': <openpathsampling.experimental.storage.storage.StorageTable at 0x11bb8d860>,
 'sample_sets': <openpathsampling.experimental.storage.storage.StorageTable at 0x11bb8d438>,
 'trajectories': <openpathsampling.experimental.storage.storage.StorageTable at 0x118c58b00>,
 'move_changes': <openpathsampling.experimental.storage.storage.StorageTable at 0x118c584a8>,
 'steps': <openpathsampling.experimental.storage.storage.StorageTable at 0x118c51390>,
 'details': <openpathsampling.experimental.storage.storage.StorageTable at 0x118c4cf28>,
 'simulation_objects': <openpathsampling.experimental.storage.storage.StorageTable at 0x118c4cef0>,
 'snapshot0': <openpathsampling.experimental.storage.storage.StorageTable at 0x118c4ce48>}

In [40]:
storage._pseudo_tables

{'volumes': <openpathsampling.experimental.storage.storage.PseudoTable at 0x118c440f0>,
 'ensembles': <openpathsampling.experimental.storage.storage.PseudoTable at 0x118c44358>,
 'pathsimulators': <openpathsampling.experimental.storage.storage.PseudoTable at 0x118c44278>,
 'pathmovers': <openpathsampling.experimental.storage.storage.PseudoTable at 0x118c44be0>,
 'networks': <openpathsampling.experimental.storage.storage.PseudoTable at 0x118c5ef98>,
 'cvs': <openpathsampling.experimental.storage.storage.PseudoTable at 0x118c5ed68>,
 'engines': <openpathsampling.experimental.storage.storage.PseudoTable at 0x118c5efd0>,
 'misc_simulation': <openpathsampling.experimental.storage.storage.PseudoTable at 0x118c55f98>}

In [41]:
storage.backend.table_to_class

{'samples': openpathsampling.sample.Sample,
 'sample_sets': openpathsampling.sample.SampleSet,
 'trajectories': openpathsampling.engines.trajectory.Trajectory,
 'move_changes': openpathsampling.movechange.MoveChange,
 'steps': openpathsampling.pathsimulators.path_simulator.MCStep,
 'details': openpathsampling.pathmover.Details,
 'simulation_objects': openpathsampling.netcdfplus.base.StorableObject,
 'snapshot0': openpathsampling.engines.toy.snapshot.ToySnapshot}

In [42]:
storage.backend.schema

{'samples': [('trajectory', 'uuid'), ('ensemble', 'uuid'), ('replica', 'int')],
 'sample_sets': [('samples', 'list_uuid')],
 'trajectories': [('snapshots', 'list_uuid')],
 'move_changes': [('mover', 'uuid'),
  ('details', 'lazy'),
  ('cls', 'str'),
  ('subchanges', 'list_uuid'),
  ('samples', 'list_uuid'),
  ('input_samples', 'list_uuid')],
 'steps': [('change', 'uuid'),
  ('active', 'uuid'),
  ('simulation', 'uuid'),
  ('mccycle', 'int')],
 'details': [('json', 'json_obj')],
 'simulation_objects': [('json', 'json_obj'), ('class_idx', 'int')],
 'snapshot0': [('velocities', 'ndarray.float32(1,2)'),
  ('coordinates', 'ndarray.float32(1,2)'),
  ('engine', 'uuid')]}

In [43]:
storage.class_info

SerializationSchema(default_info=ClassInfo(table=simulation_objects, cls=<class 'openpathsampling.netcdfplus.base.StorableObject'>, lookup_result=<class 'openpathsampling.netcdfplus.base.StorableObject'>, find_uuids=<function default_find_uuids at 0x11707b950>), class_info_list=[ClassInfo(table=simulation_objects, cls=<class 'openpathsampling.netcdfplus.base.StorableObject'>, lookup_result=<class 'openpathsampling.netcdfplus.base.StorableObject'>, find_uuids=<function default_find_uuids at 0x11707b950>), ClassInfo(table=samples, cls=<class 'openpathsampling.sample.Sample'>, lookup_result=<class 'openpathsampling.sample.Sample'>, find_uuids=<openpathsampling.experimental.storage.serialization_helpers.SchemaFindUUIDs object at 0x1174d6e80>), ClassInfo(table=sample_sets, cls=<class 'openpathsampling.sample.SampleSet'>, lookup_result=<class 'openpathsampling.sample.SampleSet'>, find_uuids=<openpathsampling.experimental.storage.serialization_helpers.SchemaFindUUIDs object at 0x1174d6f28>), 

In [44]:
storage._storage_tables

{'samples': <openpathsampling.experimental.storage.storage.StorageTable at 0x11bb8d860>,
 'sample_sets': <openpathsampling.experimental.storage.storage.StorageTable at 0x11bb8d438>,
 'trajectories': <openpathsampling.experimental.storage.storage.StorageTable at 0x118c58b00>,
 'move_changes': <openpathsampling.experimental.storage.storage.StorageTable at 0x118c584a8>,
 'steps': <openpathsampling.experimental.storage.storage.StorageTable at 0x118c51390>,
 'details': <openpathsampling.experimental.storage.storage.StorageTable at 0x118c4cf28>,
 'simulation_objects': <openpathsampling.experimental.storage.storage.StorageTable at 0x118c4cef0>,
 'snapshot0': <openpathsampling.experimental.storage.storage.StorageTable at 0x118c4ce48>}

In [45]:
storage.class_info.tables

['simulation_objects',
 'samples',
 'sample_sets',
 'trajectories',
 'move_changes',
 'steps',
 'details',
 'simulation_objects',
 'snapshot0',
 'simulation_objects']

In [46]:
for v in storage.volumes:
    print(v.name)

0.0<opC<0.2
0.0<opC<0.2
0.0<opB<0.2
0.0<opB<0.2
0.0<opA<0.4
0.0<opB<0.4
0.0<opA<0.2
0.0<opA<0.2
0.0<opC<0.4
0.0<opB<0.30000000000000004
0.0<opC<0.30000000000000004
0.0<opA<0.30000000000000004
[EmptyVolume]
[EmptyVolume]
[UnionVolume]
[UnionVolume]
[UnionVolume]
[EmptyVolume]
0.0<opC<0.5
0.0<opA<0.5
0.0<opB<0.5
[UnionVolume]
[UnionVolume]
[UnionVolume]
[UnionVolume]
[UnionVolume]
[UnionVolume]
[UnionVolume]
[UnionVolume]
[UnionVolume]
[UnionVolume]
[UnionVolume]
[UnionVolume]
[UnionVolume]
[UnionVolume]
[UnionVolume]
[UnionVolume]
[UnionVolume]
[UnionVolume]
[UnionVolume]
[UnionVolume]
[EmptyVolume]
[EmptyVolume]
[UnionVolume]
[UnionVolume]
[UnionVolume]
[UnionVolume]
[UnionVolume]
[UnionVolume]
[EmptyVolume]
[UnionVolume]
[UnionVolume]
[UnionVolume]
[UnionVolume]
[UnionVolume]
[UnionVolume]
[UnionVolume]
[UnionVolume]
[UnionVolume]
[UnionVolume]
[UnionVolume]
[UnionVolume]
[UnionVolume]
[UnionVolume]
[UnionVolume]
[UnionVolume]
[UnionVolume]
[UnionVolume]
[UnionVolume]
[UnionVolume]
[U

In [47]:
for k, v in list(storage.volumes._name_to_uuid.items()):
    print(k, v)